#### Use TF 2.0
使用keras，如果想要调整参数，不方便，使用TF2.0可以方便调整

In [25]:
from __future__ import absolute_import,division,print_function,unicode_literals
import os
import tensorflow as tf
import cProfile

In [26]:
import ssl
# 下载过程中由于证书验证不通过，取消证书验证
ssl._create_default_https_context = ssl._create_unverified_context


In [27]:
(x_train, y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

In [28]:
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]


In [29]:
dataset = tf.data.Dataset.from_tensor_slices(
(tf.cast(x_train[...,tf.newaxis]/255,tf.float32),
 tf.cast(y_train,tf.int64)))


In [30]:
dataset = dataset.shuffle(10000).batch(32)

In [31]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3)

In [32]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

In [33]:
loss_history = []

In [34]:
#build the model using sequential
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,[3,3],activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64,[3,3],activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [39]:
for epoch in range(5):
    for (batch,(images,labels)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = mnist_model(images, training=True)
            loss_value = loss(labels, logits)
        grads = tape.gradient(loss_value, mnist_model.trainable_variables)#dL/dw
        optimizer.apply_gradients(zip(grads,mnist_model.trainable_variables))#w = w-a*dL/dw
    print("Epoch {} finished".format(epoch))

Epoch 0 finished
Epoch 1 finished
Epoch 2 finished
Epoch 3 finished
Epoch 4 finished


In [40]:
x_test = tf.cast(x_test[...,tf.newaxis]/255,tf.float32)
y_test = tf.cast(y_test,tf.int64)


In [38]:
mnist_model.evaluate(x_test,y_test)

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[99.70382141113281, 0.047]